# Introducing space occupation 

We focus on the space occupation of basketball court by players. We'd like to know how we can quantify the occupation and how we can visualize this occupation of the court.

## Voronoi cutting
----

First, we can use Voronoi diagram to partitionate the court into cells which belong to a unique player.

In [1]:
from data_extracter import json_extracter
data,events=json_extracter()